In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [7]:
df = pd.read_json("/media/influx/Programming/Projects/project-final-random/Recommender/users.json")

In [8]:
df_train = df.drop(['dateOfBirth', 'name', 'email', 'profile_pic', 'connectionId'], axis=1)

In [9]:
df_train

,id,universityLocation,field,workplace,specialties
0,1,Tehran,AA,Tesla,"[Database, Front-End, Crypto]"
1,2,Princeton,BB,Apple,"[Full-Stack, Machine Learning, Fast Type]"
2,3,Cambridge,AA,Meta,"[Blockchain, C++, AI]"
3,4,Cambridge,BB,Intel,"[Photoshop, Game, Django]"
4,5,Princeton,DD,AMD,"[Full-Stack, Machine Learning, Photoshop, Game..."
...,...,...,...,...,...
1996,1997,ETH,AA,Digikala,"[Verilog, Crypto, C#, Front-End, Full-Stack]"
1997,1998,Carnegie,DD,AMD,"[PHP, Machine Learning, Python, Database, Data..."
1998,1999,Cambridge,AA,AMD,"[Machine Learning, Data, Hardware, Django]"
1999,2000,Stanford,CC,Digikala,"[Photoshop, Machine Learning, C++]"


In [10]:
df_train = pd.get_dummies(df_train, columns=["field"])
df_train = pd.get_dummies(df_train, columns=["universityLocation"])
df_train = pd.get_dummies(df_train, columns=["workplace"])

In [11]:
df_train

,id,specialties,field_AA,field_BB,field_CC,field_DD,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,...,workplace_Digikala,workplace_Google,workplace_Intel,workplace_Maghdonie,workplace_Meta,workplace_Microsoft,workplace_NVIDIA,workplace_Snapp,workplace_TesUberla,workplace_Tesla
0,1,"[Database, Front-End, Crypto]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,"[Full-Stack, Machine Learning, Fast Type]",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,"[Blockchain, C++, AI]",1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,4,"[Photoshop, Game, Django]",0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,5,"[Full-Stack, Machine Learning, Photoshop, Game...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,1997,"[Verilog, Crypto, C#, Front-End, Full-Stack]",1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1997,1998,"[PHP, Machine Learning, Python, Database, Data...",0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1998,1999,"[Machine Learning, Data, Hardware, Django]",1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1999,2000,"[Photoshop, Machine Learning, C++]",0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [15]:
spec = set(s for user in df_train['specialties'] for s in user)
for special in spec:
    df_train[special] = [0 for i in range(2001)]

In [16]:
i = 0
for user in df_train['specialties']:
    for s in user:
        df_train.at[i, s] = 1
    i+=1
    
df_train = df_train.drop(['specialties', 'id'], axis=1)

In [17]:
df_train

,field_AA,field_BB,field_CC,field_DD,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,...,Python,PHP,Front-End,IT,C#,Machine Learning,Verilog,Crypto,Java,Full-Stack
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,1,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,1,1,0,1
1997,0,0,0,1,0,0,0,1,0,0,...,1,1,0,0,0,1,0,0,0,0
1998,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1999,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [18]:
df_train.columns

Index(['field_AA', 'field_BB', 'field_CC', 'field_DD',
       'universityLocation_Amirkabir', 'universityLocation_Berkeley',
       'universityLocation_Cambridge', 'universityLocation_Carnegie',
       'universityLocation_ETH', 'universityLocation_Esfahan',
       'universityLocation_Harvard', 'universityLocation_MIT',
       'universityLocation_Oroomie', 'universityLocation_Oxford',
       'universityLocation_Princeton', 'universityLocation_Sharif',
       'universityLocation_Stanford', 'universityLocation_Tehran',
       'workplace_AMD', 'workplace_Adobe', 'workplace_Amazon',
       'workplace_Apple', 'workplace_Bazaar', 'workplace_Cisco',
       'workplace_Digikala', 'workplace_Google', 'workplace_Intel',
       'workplace_Maghdonie', 'workplace_Meta', 'workplace_Microsoft',
       'workplace_NVIDIA', 'workplace_Snapp', 'workplace_TesUberla',
       'workplace_Tesla', 'Django', 'Algorithm', 'Fast Type', 'Hardware',
       'Game', 'Software', 'Data', 'Database', 'Back-End', 'Photosho

In [19]:
ability_power = list()
for user in df_train.values:
    if(user[0] == 1):
        power = 4
    elif user[1] == 1:
        power = 3
    elif user[2] == 1:
        power = 2
    elif user[3] == 1:
        power = 1
    
    ability = 0
    for i in range(33, 55):
        if(user[i] == 1):
            ability += 1
            
    ability_power.append(ability/power)
    

In [20]:
df_train['ability_power'] = ability_power

In [21]:
df_train = df_train.drop(['field_AA', 'field_BB', 'field_CC', 'field_DD', 'Algorithm', 'Database', 'Game'], axis=1)

In [22]:
df_train = df_train.drop(['Python', 'C#', 'Software', 'Back-End', 'Fast Type', 'Verilog'], axis=1)

In [23]:
df_train = df_train.drop(['Photoshop', 'AI', 'Java', 'PHP', 'Data', 'Front-End', 'Hardware', 'Django'], axis=1)

In [24]:
df_train = df_train.drop(['C++', 'Crypto', 'Full-Stack', 'IT', 'Machine Learning', 'Blockchain'], axis=1)

In [25]:
df_train

,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,universityLocation_Harvard,universityLocation_MIT,universityLocation_Oroomie,universityLocation_Oxford,...,workplace_Google,workplace_Intel,workplace_Maghdonie,workplace_Meta,workplace_Microsoft,workplace_NVIDIA,workplace_Snapp,workplace_TesUberla,workplace_Tesla,ability_power
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.000000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.666667
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.750000
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1.000000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000
1997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.000000
1998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000
1999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.500000


In [26]:
dsc = df_train['ability_power'].values
sc = StandardScaler()
dsc = dsc.reshape(-1,1)
dsc = sc.fit_transform(dsc)

In [27]:
dsc

array([[-0.8943157 ],
       [-1.09880955],
       [-1.04768609],
       ...,
       [-0.8943157 ],
       [-0.58757492],
       [-1.35442687]])

In [28]:
df_train["ability_power"] = dsc

In [29]:
df_train

,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,universityLocation_Harvard,universityLocation_MIT,universityLocation_Oroomie,universityLocation_Oxford,...,workplace_Google,workplace_Intel,workplace_Maghdonie,workplace_Meta,workplace_Microsoft,workplace_NVIDIA,workplace_Snapp,workplace_TesUberla,workplace_Tesla,ability_power
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-0.894316
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.098810
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,-1.047686
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,-0.894316
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.559611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.894316
1997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.173092
1998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.894316
1999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.587575


In [30]:
X = df_train.values

In [31]:
def estimate_gaussian(X): 
    m, n = X.shape
    
    mu = np.mean(X, axis=0)
    var = np.var(X, axis=0)
        
    return mu, var

In [32]:
def ProbabilityAnomaly(X):
    mu , var = estimate_gaussian(X)
    
    P_X = dict()
    idx = 0
    for user in X:
        p_x_i = (1/np.sqrt(2*np.pi*var))*np.exp(-(np.power(user-mu, 2))/(2*var))
        P_X[idx] = np.prod(p_x_i)
        idx += 1
    
    return P_X

In [33]:
print((ProbabilityAnomaly(X)))

{0: 125.7034409830186, 1: 18.19739513266698, 2: 35.86746287223403, 3: 3.415137339264444, 4: 46.483315195499166, 5: 8.867630832409583, 6: 1.8450442053717007, 7: 0.7071082161391316, 8: 75.85829708273445, 9: 26.278228676386032, 10: 185.0801419764359, 11: 4.994999794318004, 12: 105.14976597338574, 13: 8.438568724174566, 14: 21.921307935138618, 15: 6.136692287529795, 16: 70.0712230895395, 17: 73.23497050921696, 18: 87.95192335406637, 19: 0.4151020338973867, 20: 31.844755310336602, 21: 439.3319297186166, 22: 92.24494001933765, 23: 1.0019461935158365, 24: 5.636530409651378, 25: 17.6383777647657, 26: 1.8182082200221084, 27: 2.408338897706364, 28: 284.81514960441837, 29: 111.37075586562472, 30: 5.1727512860853855, 31: 2.545327056372128, 32: 71.53811819940451, 33: 11.574945563684762, 34: 59.16161032224626, 35: 135.03785575538106, 36: 7.030183281742678, 37: 310.3061193338032, 38: 10.045235339238108, 39: 6.103087840316488, 40: 139.42325846033043, 41: 2.8641244157461525, 42: 113.94200561558873, 43:

In [34]:
x = ProbabilityAnomaly(X)

In [35]:
x = dict(sorted(x.items(), key=lambda item: item[1]))

In [36]:
x

{2000: 0.0,
 311: 0.0015988087991200357,
 1557: 0.0042850341968407165,
 805: 0.009264710583601482,
 1306: 0.009375075061021747,
 402: 0.012579837050968578,
 1317: 0.013377952048278421,
 405: 0.017303424104151574,
 97: 0.028813682128517003,
 1365: 0.03483561964440515,
 1797: 0.04241679411152797,
 1991: 0.04241679411152801,
 473: 0.05030154422738132,
 1345: 0.056017392625488056,
 423: 0.05669366071798188,
 1764: 0.058464386232785034,
 1042: 0.0636511791505219,
 339: 0.0643236047863226,
 515: 0.06848835292220566,
 422: 0.08460100593839721,
 312: 0.08887965063105506,
 1048: 0.09191031007479648,
 776: 0.0923232083607782,
 952: 0.1029282174529303,
 742: 0.10598849955318235,
 1088: 0.10946584036784743,
 212: 0.1332885724590263,
 1712: 0.13663851474023125,
 257: 0.13698966330605714,
 460: 0.13698966330605714,
 1081: 0.15343148269014603,
 1334: 0.15728767682329042,
 906: 0.16117342892379743,
 1846: 0.1924645259935555,
 116: 0.19246452599355568,
 1085: 0.20001452212535398,
 1581: 0.2000145221253